In [1]:
from rainpy.raindrop import make_frame

from collections import namedtuple

drops = {"name": "example", 
                "floder": "./example", 
                "author": "zmdsn",
                "email":"zmdsn@126.com",
                "url":"https://github.com/zmdsn/rainpy"}
RainDrop = namedtuple("RainDrop", drops.keys())
drop = RainDrop(**drops)

make_frame(drop)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 20.4 kB/s eta 0:00:0000:0200:15
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 23.1 kB/s eta 0:00:00a 0:00:10
